# Store all from category Beruf

In [1]:
#store pages from category Beruf
from wikitools import wiki, api,category
import json

def seen_wiki_categories(lang,category_page,overloop=0,n=6):
    global seen_categories_
    global seen_pages_
    where_go_categories_=[]
    #print "We are here: "+ category_page
    seen_categories_.append(category_page)
    site = wiki.Wiki("http://"+lang+".wikipedia.org/w/api.php") 
    cat = category.Category(site, category_page) # Create object for "Category:xxxxx"
    for article in cat.getAllMembersGen(titleonly=True,namespaces=[0]): # iterate through all the pages in ns 0
        if article not in seen_pages_ :
            seen_pages_.append(article)
        else:
            continue
    for subcategory in cat.getAllMembersGen(titleonly=True,namespaces=[14]):
        if subcategory not in seen_categories_ :
            where_go_categories_.append(subcategory)
        else:
            continue
    for c in where_go_categories_ :
        if c not in seen_categories_:
            if overloop<n:
                seen_wiki_categories(lang,c,overloop=overloop+1,n=n)
            else:
                print "overloop ==="+c
                pass


In [15]:

seen_categories_=[]
seen_pages_=[]
seen_wiki_categories("de","Kategorie:Beruf",overloop=0,n=3)
seen_pages_=list(set(seen_pages_))
print len(seen_pages_)
with open('de/wiki/berufe_wiki4.json', 'w') as f: 
    json.dump(seen_pages_,  f,  indent=4)

overloop ===Kategorie:Amtsbezeichnung (Deutschland)
overloop ===Kategorie:Amtsbezeichnung (Österreich)
overloop ===Kategorie:Dienstgrad
overloop ===Kategorie:Dienststellung
overloop ===Kategorie:Polizeiberuf
overloop ===Kategorie:Leiter der litauischen Zollverwaltung
overloop ===Kategorie:Mitglied des Zoll-Ski-Teams
overloop ===Kategorie:Anwaltschaft in der Schweiz
overloop ===Kategorie:Anwaltschaft in Russland
overloop ===Kategorie:Anwaltskanzlei
overloop ===Kategorie:Anwaltschaft in Deutschland
overloop ===Kategorie:Anwaltschaft in Frankreich
overloop ===Kategorie:Anwaltschaftsgeschichte
overloop ===Kategorie:Anwaltschaft in Litauen
overloop ===Kategorie:Rechtsanwalt
overloop ===Kategorie:Rechtsanwaltskammer
overloop ===Kategorie:Anwaltschaft im Vereinigten Königreich
overloop ===Kategorie:Militärisches Luftfahrtpersonal
overloop ===Kategorie:Binnenschifffahrtsberuf
overloop ===Kategorie:Seefahrtsberuf
overloop ===Kategorie:Fernsehkoch
overloop ===Kategorie:Industrieller Elektroberuf

# Levenshtein dictance

In [1]:
#compare seen_categories_ and beruf
import json
def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)
words=load_simple_json('de/occupation_all.json')
wiki_pages=load_simple_json('de/wiki/berufe_wiki4.json')
print len(words)#4240#4239#4252

both=[]
for i in words:
    if i in wiki_pages:
        both.append(i)
    if words[i][1] in wiki_pages:
        both.append(i)
print "we have ",len(both),"wiki pages from category Beruf and our list of occupations"#640 and 648(+for feminine)#710#708#711

4252
we have  711 wiki pages from category Beruf and our list of occupations


In [2]:
both2={}
for i in words:
    for j in wiki_pages:
        if i in j:
            if i not in both2.keys():
                both2[i]=[j]
            else:
                both2[i].append(j)
                #both2[i]=[a]
        #if words[i][1] in j:
        #    both2.append(i)
print len(both2)#800#799#802#801

801


In [3]:
m_no_page=load_simple_json('de/wiki/m_no_page.json')

#match no_page and wiki_berufe4
both3={}
for i in m_no_page:
    for j in wiki_pages:
        if i in j:
            if i not in both3.keys():
                both3[i]=[j]
            else:
                both3[i].append(j)
print "# of Masculine_no_pages in list of pages from category Beruf (substring inclusion):",len(both3)#17#17#12#16

#match wiki_berufe4 and no_page
both4={}
for i in m_no_page:
    for j in wiki_pages:
        if j in i:
            if j not in both4.keys():
                both4[j]=[i]
            else:
                both4[j].append(i)
print "# of Masculine_no_pages in list of pages from category Beruf (substring inclusion reversed):",len(both4)#206#209#204#209

# of Masculine_no_pages in list of pages from category Beruf (substring inclusion): 16
# of Masculine_no_pages in list of pages from category Beruf (substring inclusion reversed): 209


In [4]:
#levenshtein and string inclusion
import Levenshtein

print "With Levenshtein distance<=2 or ratio>0.8"
m_page=load_simple_json('de/wiki/m_page.json')
page_m=m_page
both6={}
k=0
m=0
for i in m_no_page:
    for j in wiki_pages: 
                    if i in j:
                        if i not in both6.keys():
                            both6[i]=[j]
                        else:
                            both6[i].append(j)
                    elif Levenshtein.ratio(i, j)>0.8:
                        print "===>",i,"==",j
                        m+=1
                        if i not in both6.keys():
                            both6[i]=[j]
                        else:
                            both6[i].append(j)
                    elif Levenshtein.distance(i, j)<=2:
                        k+=1
                        print i,":",j    
                        if i not in both6.keys():
                            both6[i]=[j]
                        else:
                            both6[i].append(j)
print k#30#30#30
print m#498#518#517

With Levenshtein distance<=2 or ratio>0.8
===> Planungstechniker == Zerspanungstechniker
===> Metallformer == Metallverformer
===> Fachpraktiker für Land- und  Baumaschinentechnik == Mechaniker für Land- und Baumaschinentechnik
===> Farb- und Oberflächengestalter == Garten- und Grünflächengestalter
===> Abfallberater == Abfallbeauftragter
===> Technischer Leiter == Technischer Zeichner
===> Medizinisch-technischer Laboratoriumsassistent == Medizinisch-technischer Assistent
===> Elektroinstallateurmeister == Elektroinstallateur
===> Meliorationstechniker == Informationstechniker
===> Betonstein- und Terrazzoherstellermeister == Betonstein- und Terrazzohersteller
===> Brandschutzglaser == Brandschutzwart
===> Brandschutzglaser == Brandschutzgutachter
===> Holzblasinstrumentenfacharbeiter == Holzblasinstrumentenmacher
===> Fachverkäufer im Lebensmittelhandwerk (Konditorei) == Fachverkäufer im Lebensmittelhandwerk
===> Fachverkäufer im Lebensmittelhandwerk (Konditorei) == Fachverkäufer im 

In [5]:
#lets check both3 values
page_m=load_simple_json('de/wiki/m_page.json')
page_f=load_simple_json('de/wiki/f_page.json')
m_links_to_feminine=load_simple_json('de/wiki/m_links_to_feminine.json')
m_links_to_smth=load_simple_json('de/wiki/m_links_to_smth.json')
print "======Masculine_no_pages in list of pages from category Beruf (substring inclusion):======",len(both3)
for i in both3:
    for j in both3[i]:
        if j in m_page:
            print "!!!already in pages m",j
        elif j in m_links_to_feminine:
            print "!!!in m links to feminine",j
        elif j in m_links_to_smth:
            print "!!!in m links to smth",j
        elif j in m_links_to_smth.values():
            print "!!!in m links to smth_values",j
        elif j in page_f:
            print "!!! in feminine pages",j
        elif i in both6.keys():
            print "=== in levenstein values:",i," :",both6[i],both3[i]
        else:
            print "++Rest:",i," :",j

======Masculine_no_pages in list of pages from category Beruf (substring inclusion):====== 16
=== in levenstein values: Fertigungstechnischer Assistent  : [u'Informationstechnischer Assistent', u'Operationstechnischer Assistent', u'Konstruktions- und Fertigungstechnischer Assistent', u'Datentechnischer Assistent', u'Gestaltungstechnischer Assistent'] [u'Konstruktions- und Fertigungstechnischer Assistent']
!!!already in pages m Fachverkäufer im Lebensmittelhandwerk
!!!already in pages m Fachverkäufer im Lebensmittelhandwerk (Fleischerei)
!!!already in pages m Fachverkäufer im Lebensmittelhandwerk (Bäckerei)
!!!already in pages m Betonstahlbieger und -flechter
!!!already in pages m Zahnmedizinischer Fachassistent
!!!already in pages m Fachangestellter für Bürokommunikation
!!!already in pages m Fachangestellter für Bäderbetriebe
!!!already in pages m Fachangestellter für Medien- und Informationsdienste
!!!already in pages m Fachangestellter für Arbeitsmarktdienstleistungen
!!!already in 

In [6]:
from collections import defaultdict
m_page=load_simple_json('de/wiki/m_page.json')
m_links_to_feminine=load_simple_json('de/wiki/m_links_to_feminine.json')
m_links_to_feminine_lev=load_simple_json('de/wiki/m_links_to_feminine_lev.json')
m_links_to_smth=load_simple_json('de/wiki/m_links_to_smth.json')
substr_from_both4=defaultdict(list)
for i in both4:
    for j in both4[i]:
        if j in m_page:
            print "!!!already in pages",j
        elif j in m_links_to_feminine:
            print "!!!links to feminine",j
        elif j in m_links_to_smth:
            print "!!!links to smth",j
        elif j in m_links_to_smth.values():
            print "!!!links to smth_values",j
        elif j in page_f:
            print "!!!links to feminine",j
    if i in m_page:
        pass
    elif i in m_links_to_smth.keys():
        pass
    elif i in m_links_to_feminine:
        pass
    elif i in m_links_to_feminine_lev.keys():
        pass
    elif i in m_links_to_smth.values():
        print i
    else:
        substr_from_both4[i]=both4[i]
        #print i
        #else: 
        #    print i
with open('de/wiki/berufe_substring_19.01.16.json', 'w') as f: #m_no_page:[wiki_pages_in_beruf_category_withsubstring_inclusion]
    json.dump(substr_from_both4,  f,  indent=4)

Metallbau
Fahrer
Heizungsbauer
Polstern
Systemadministrator
Model
Fliesenleger


In [7]:

def check_pages_in_list(both6):
    new_both6={}
    for i in both6:
        if i not in new_both6.keys():
            new_both6[i]=both6[i]
        for j in both6[i]:
            if j in page_m:
                index=int(both6[i].index(j))
                new_both6[i][index]=both6[i][index],1 #1=page exist
            elif j in m_links_to_feminine:
                index=int(both6[i].index(j))
                #new_both6[i]=both6[i]
                new_both6[i][index]=both6[i][index],2 #2=redirects to feminine
            elif j in m_links_to_smth:
                index=int(both6[i].index(j))
                #new_both6[i]=both6[i]
                new_both6[i][index]=both6[i][index],3 #3=redirects to smth
            elif j in m_links_to_smth.values():
                index=int(both6[i].index(j))
                #new_both6[i]=both6[i]
                new_both6[i][index]=both6[i][index],4 #4=redirects to smth value
            else:
                index=int(both6[i].index(j))
                #new_both6[i]=both6[i]
                new_both6[i][index]=both6[i][index],0 #no page
    return new_both6
new_both6=check_pages_in_list(both6)


In [8]:
with open('de/wiki/berufe_levenshtein_.json', 'w') as f: #m_no_page:[wiki_pages_in_beruf_category]
    json.dump(new_both6,  f,  indent=4)
with open('de/wiki/berufe_substring_.json', 'w') as f: #m_no_page:[wiki_pages_in_beruf_category_withsubstring_inclusion]
    json.dump(both4,  f,  indent=4)
    


# Check all the values manually:

Read all values from files berufe_levenstein and berufe_substring_ and then manually identify wheather those professions correspond to professions in list all_occupations. Encoding of manual assignment: 99 - the same profession as in our list, but with different writing (always masculine); 100 - neutral form of profession, e.g. name of field ( "Audiodesign"); 98 - female profesion corespondent to existent male profesion (We found no evidence). Then we save data in the file berufe_levenshtein_3105.json

# Read all with mark 99 and 100 and add 99 into list of existed pages 

print all neutral forms and then save them into the file n_page_levenshtein.json
save assigned new names of male profesion names into the file m_page_levenshtein.json
creat and save correspondent female labels for found professions into the file m_page_levenshtein_feminine_form.json . We will use female labels in order to check wheather they exist on wikipedia too. 

In [50]:
lev_corrected=load_simple_json('de/wiki/berufe_levenshtein_19.01.16.json')#old was ..._3105.json

new_page_m={}
new_page_n={}
for i in lev_corrected:
    for j in lev_corrected[i]:
        if j[1]==99:
            new_page_m[i]=j[0]
            
        elif j[1]==98:
            print i,"==",j[0], "?????"
        elif j[1]==100:
            print i, "is =====>",j[0],"Neutral?"
            new_page_n[i]=j[0]
print len(new_page_m) ,   len(new_page_n)       

Audiodesigner is =====> Audiodesign Neutral?
Dekorationenmaler is =====> Dekorationsmalerei Neutral?
Medientechniker is =====> Medientechnik Neutral?
Projektassistent is =====> Projektassistenz Neutral?
Messebauer is =====> Messebau Neutral?
Filmgeschäftsführer is =====> Filmgeschäftsführung Neutral?
Pflegedienstleiter is =====> Pflegedienstleitung Neutral?
29 7


In [51]:
with open('de/wiki/m_page_levenshtein.json', 'w') as f:
    json.dump(new_page_m,  f,  indent=4)
with open('de/wiki/m_neutral_lev.json', 'w') as f: 
    json.dump(new_page_n,  f,  indent=4)

# Retrieve values from berufe_substrin_...-cor.json

In [52]:
import json
def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)
berufe_substring=load_simple_json('de/wiki/berufe_substring_19.01.16_cor.json')
m_no_page=load_simple_json('de/wiki/m_no_page.json')
m_page_levenshtein=load_simple_json('de/wiki/m_page_levenshtein.json')
print "We had ",len(m_no_page),len(m_page_levenshtein)
for i in berufe_substring:
    if 1 in berufe_substring[i]:
        print i,berufe_substring[i][0]
        m_page_levenshtein[berufe_substring[i][0]]=i
        if berufe_substring[i][0] in m_no_page:
            m_no_page.remove(berufe_substring[i][0])
print "We have now ",len(m_no_page),len(m_page_levenshtein)

with open('de/wiki/m_no_page.json', 'w') as f:
    json.dump(m_no_page,  f,  indent=4)
with open('de/wiki/m_page_levenshtein.json', 'w') as f: 
    json.dump(m_page_levenshtein,  f,  indent=4)

We had  2836 29
Syndikus Syndikus-Anwalt
Sonnenschutztechniker Rollladen- und Sonnenschutztechnikermeister
Werkschutz Werkschutzleiter
We have now  2836 32


# Build feminine form of words

In [32]:
new_page_m=load_simple_json('de/wiki/m_page_levenshtein.json')
new_page_m_feminine_form={}
for i in new_page_m:
    if i=="Syndikus-Anwalt":
        new_page_m_feminine_form[i]="Syndika"
    elif i=="Wekschutzleiter":
        new_page_m_feminine_form[i]="Wekschutzleiterin"
        #Werkschutzin?????
    elif len(new_page_m[i].split(" "))==1:
        if new_page_m[i][-4:]!="mann":
            new_page_m_feminine_form[i]=new_page_m[i]+"in"
        else:
            new_page_m_feminine_form[i]=new_page_m[i].replace("mann","frau")
    else:
        if ("Verwaltungsbetriebswirt"not in new_page_m[i])&("Fachberater"not in new_page_m[i]):
            new_page_m_feminine_form[i]=new_page_m[i].replace("er ","e ")+"in"
        else:
            m=new_page_m[i].split(" ")
            new_page_m_feminine_form[i]=new_page_m[i].replace(m[0],m[0]+"in ")
new_page_m_feminine_form #check this feminine pages in wiki?


{u'Automechatroniker': u'Automobil-Mechatronikerin',
 u'Bezirksleiter': u'Bezirksamtsleiterin',
 u'B\xfcchsenmachermeister': u'B\xfcchsenmeisterin',
 u'Chemisch-technischer Assistent': u'Chemisch-Technische Assistentin',
 u'Einrichtungsfachberater': u'Einrichtungsberaterin',
 u'Film-Regisseur': u'Filmregisseurin',
 u'Foto- und medientechnischer Assistent': u'Gestaltungs- und medientechnische Assistentin',
 u'Gartenbauwerker': u'Gartenbaufachwerkerin',
 u'Handelslehrer': u'Diplom-Handelslehrerin',
 u'IT-System-Kaufmann': u'IT-Systemkauffrau',
 u'Kalkulator': u'Calculatorin',
 u'Kettengoldschmied': u'Kettenschmiedin',
 u'Kommunikationstechnischer Assistent': u'Informationstechnische Assistentin',
 u'Kunststoff-Formgeber': u'Kunststoffformgeberin',
 u'Logistikmeister': u'Gepr\xfcfte Logistikmeisterin',
 u'Maschinentechniker': u'Maschinenbautechnikerin',
 u'Mathematisch-technischer Assistent': u'Mathematisch-Technische Assistentin',
 u'Medienberater': u'Mediaberaterin',
 u'Medienwirt': u'M

In [33]:

with open('de/wiki/m_page_levenshtein_feminine_form.json', 'w') as f: 
    json.dump(new_page_m_feminine_form,  f,  indent=4)
    

## del exiting pages with levenstain dist from m_no_page

In [49]:
import json

def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)



m_page_levenshtein=load_simple_json('de/wiki/m_page_levenshtein.json')
m_no_page=load_simple_json('de/wiki/m_no_page.json')
#n_page_levenshtein=load_simple_json('de/wiki/n_page_levenshtein.json')
print len(m_no_page)
for i in m_page_levenshtein:
    if i in m_no_page:
        m_no_page.remove(i)

print len(m_no_page)

with open('de/wiki/m_no_page.json', 'w') as f: 
    json.dump(m_no_page,  f,  indent=4)

2836
2836


# Check if page for feminine form exist

save existent redirections into f_links_to_mascuilne_lev.json

In [35]:
from bs4 import BeautifulSoup
from wikitools import wiki, api
import json
import re


def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)

    
def check_revision(revisions):
    redir=""
    page=False
    for j in revisions:
        redir_marker=["#WEITERLEITUNG","#REDIRECT","#redirect","#weiterleitung","#Weiterleitung","#Redirect"]
        if ("#WEITERLEITUNG" in j["*"])|("#REDIRECT" in j["*"])|("#redirect" in j["*"])|("#weiterleitung" in j["*"])\
        |("#Redirect" in j["*"])|("#Weiterleitung" in j["*"]):
            m = re.search(r"\[\[(.+)\]\]", j["*"])
            s = m.group(0)[2:].split('#', 1)[0].replace("]]",'')
            redir=s
        else:
            page=True
    return redir,page


    
new_page_m_feminine_form=load_simple_json('de/wiki/m_page_levenshtein_feminine_form.json')

redirection_f_new={}
page_f_new={}
no_page_f_new=[]

for i in new_page_m_feminine_form:
    beruf_f=new_page_m_feminine_form[i]
    l="de" 
    site = wiki.Wiki("http://"+l+".wikipedia.org/w/api.php")
    #last revision
    params={'action':'query','titles':beruf_f,'prop':'revisions','rvprop':'timestamp|content','format':'json','continue':''}
    request = api.APIRequest(site, params)
    result=request.query()
    
    if result["query"]["pages"].values()[0].get("revisions"):
        redir,page=check_revision(result["query"]["pages"].values()[0]["revisions"])
        if redir:
            redirection_f_new[i]=redir
        if page:
            page_f_new[i]=beruf_f        
    else:

        no_page_f_new.append(beruf_f)

print page_f_new
print redirection_f_new

#check redirections
new_page_m=load_simple_json('de/wiki/m_page_levenshtein.json')
words=load_simple_json('de/occupation_all.json')
f_links_to_mascuilne_lev={}
for i in redirection_f_new:
    if redirection_f_new[i]==new_page_m[i]:
        f_links_to_mascuilne_lev[words[i][1]]={new_page_m_feminine_form[i]:redirection_f_new[i]}
    else:
        print i
with open('de/wiki/f_links_to_mascuilne_lev.json', 'w') as f: 
    json.dump(f_links_to_mascuilne_lev,  f,  indent=4)
    

    

{}
{u'Film-Regisseur': u'Filmregisseur', u'Video-Journalist': u'Videojournalist', u'Syndikus-Anwalt': u'Syndikus', u'IT-System-Kaufmann': u'IT-Systemkaufmann', u'Mathematisch-technischer Assistent': u'Mathematisch-Technischer Assistent'}


No feminine page exist. All the redirectins were to masculine form.

## del from f_no_page all values from f_links_to_mascuilne_lev

In [36]:
import json

def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)



f_links_to_mascuilne_lev=load_simple_json('de/wiki/f_links_to_mascuilne_lev.json')
f_no_page=load_simple_json('de/wiki/f_no_page.json')
#n_page_levenshtein=load_simple_json('de/wiki/n_page_levenshtein.json')
print len(f_no_page)
for i in f_links_to_mascuilne_lev:
    if i in f_no_page:
        f_no_page.remove(i)

print len(f_no_page)

with open('de/wiki/f_no_page.json', 'w') as f: 
    json.dump(f_no_page,  f,  indent=4)

3941
3940


## del from m_no_page and f_no_page where exist corespondent neutral pages?

In [8]:
import json
def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)
m_no_page=load_simple_json('de/wiki/m_no_page.json')
f_no_page=load_simple_json('de/wiki/f_no_page.json')
n_page_levenshtein=load_simple_json('de/wiki/n_page_levenshtein.json')   
words=load_simple_json('de/occupation_all.json')#{masculine:[[],[feminine]]}
print "We had:",len(m_no_page)
print "We had:", len(f_no_page)
for p in n_page_levenshtein:
    if p in m_no_page:
        m_no_page.remove(p)
    else:
        print p
    if words[p][1] in f_no_page:
        f_no_page.remove(words[p][1])
    else:
        print p
print "Now:",len(m_no_page)
print "Now:",len(f_no_page)

with open('de/wiki/m_no_page.json', 'w') as f: 
    json.dump(m_no_page,  f,  indent=4)
with open('de/wiki/f_no_page.json', 'w') as f: 
    json.dump(f_no_page,  f,  indent=4)

We had: 2830
We had: 3932
Now: 2823
Now: 3925


In [ ]:
#TODO check the next and add f_links_to_mascuilne_with_link to the f_redirection value. DONE

In [3]:
import json
def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)

m_links_to_feminine_lev={}    

with open('de/wiki/m_links_to_feminine_lev.json', 'w') as f: 
    json.dump(m_links_to_feminine_lev,  f,  indent=4)

words=load_simple_json('de/occupation_all.json')#{masculine:[[],[feminine]]}
words_revert={words[i][1]:i for i in words}#feminine:masculine
    
    
m_links_to_feminine=load_simple_json('de/wiki/m_links_to_feminine.json')
m_links_to_feminine_lev=load_simple_json('de/wiki/m_links_to_feminine_lev.json')
m_links_to_smth=load_simple_json('de/wiki/m_links_to_smth.json')


f_links_to_mascuilne=load_simple_json('de/wiki/f_links_to_mascuilne.json')
f_links_to_smth=load_simple_json('de/wiki/f_links_to_smth.json')
#redirection_n=load_simple_json('de/wiki/n_redirects.json')
f_links_to_mascuilne_lev=load_simple_json('de/wiki/f_links_to_mascuilne_lev.json')
f_links_to_mascuilne_with_link=load_simple_json('de/wiki/f_links_to_mascuilne_with_link.json')


def merge_dicts(*dict_args):
    '''
    Given any number of dicts, shallow copy and merge into a new dict,
    precedence goes to key value pairs in latter dicts.
    '''
    result = {}
    for dictionary in dict_args:
        result.update(dictionary)
    return result

m_red={}
for m in m_links_to_feminine:
    m_red[m]=words[m][1]
    
f_red={}
for f in f_links_to_mascuilne:
    f_red[f]=words_revert[f]

    
m_redirection = merge_dicts(m_red, m_links_to_feminine_lev, m_links_to_smth) 
f_redirection = merge_dicts(f_red,f_links_to_smth,f_links_to_mascuilne_lev,f_links_to_mascuilne_with_link)

with open('de/wiki/m_redirection.json', 'w') as f: 
    json.dump(m_redirection,  f,  indent=4)

with open('de/wiki/f_redirection.json', 'w') as f: 
    json.dump(f_redirection,  f,  indent=4)

# Table

In [4]:
from prettytable import PrettyTable

import json
def load_simple_json(filename):
    with open(filename, 'r') as f:
        return json.load(f)
    
words=load_simple_json('de/occupation_all.json')#{masculine:[[],[feminine]]}
words_revert={words[i][1]:i for i in words}#feminine:masculine
neutral=load_simple_json('de/neutral_cleaned.json')

no_page_m=load_simple_json('de/wiki/m_no_page.json')
no_page_f=load_simple_json('de/wiki/f_no_page.json')
no_page_n=load_simple_json('de/wiki/n_no_page.json')
page_m=load_simple_json('de/wiki/m_page.json')
page_f=load_simple_json('de/wiki/f_page.json')
page_n=load_simple_json('de/wiki/n_page.json')
m_page_levenshtein=load_simple_json('de/wiki/m_page_levenshtein.json')
#n_page_levenshtein=load_simple_json('de/wiki/n_page_levenshtein.json')


page_f_validated=load_simple_json('de/wiki/f_page_validated.json')
page_f_ambigious=load_simple_json('de/wiki/f_page_ambigious.json')
page_f_other=load_simple_json('de/wiki/f_page_other.json')  
    
page_m_validated=load_simple_json('de/wiki/m_page_validated.json') 
page_m_ambigious=load_simple_json('de/wiki/m_page_ambigious.json')
page_m_other=load_simple_json('de/wiki/m_page_other.json')

page_n_validated=load_simple_json('de/wiki/n_page_validated.json')
page_n_ambigious=load_simple_json('de/wiki/n_page_ambigious.json')
page_n_other=load_simple_json('de/wiki/n_page_other.json') 

m_links_to_feminine=load_simple_json('de/wiki/m_links_to_feminine.json')
m_links_to_feminine_lev=load_simple_json('de/wiki/m_links_to_feminine_lev.json')
m_links_to_smth=load_simple_json('de/wiki/m_links_to_smth.json')
f_links_to_mascuilne=load_simple_json('de/wiki/f_links_to_mascuilne.json')
f_links_to_smth=load_simple_json('de/wiki/f_links_to_smth.json')
redirection_n=load_simple_json('de/wiki/n_redirects.json')
n_links_to_masculine=load_simple_json('de/wiki/n_links_to_masculine.json')
n_links_to_smth=load_simple_json('de/wiki/n_links_to_smth.json')

f_links_to_mascuilne_lev=load_simple_json('de/wiki/f_links_to_mascuilne_lev.json')
f_links_to_mascuilne_with_link=load_simple_json('de/wiki/f_links_to_mascuilne_with_link.json')
f_redirection=load_simple_json('de/wiki/f_redirection.json')
m_redirection=load_simple_json('de/wiki/m_redirection.json')

t = PrettyTable(['-de-',"Occup.", 'Wiki pages', "Validated pages","Ambigious pages","Redirects",
                 "Redir. to oposit gender"])
t.add_row(['Masculine',len(words), len(page_m)+len(m_page_levenshtein), len(page_m_validated)+len(m_page_levenshtein),
           len(page_m_ambigious),
           len(m_redirection), len(m_links_to_feminine)])
t.add_row(['Feminine',len(words_revert), len(page_f), len(page_f_validated), len(page_f_ambigious),
           len(f_redirection),
           len(f_links_to_mascuilne)+len(f_links_to_mascuilne_lev)])
t.add_row(['Neutral',len(neutral), len(page_n), len(page_n_validated), len(page_n_ambigious),
           len(redirection_n), "to M:"+str(len(n_links_to_masculine))+", to F:0"])
print t

+-----------+--------+------------+-----------------+-----------------+-----------+-------------------------+
|    -de-   | Occup. | Wiki pages | Validated pages | Ambigious pages | Redirects | Redir. to oposit gender |
+-----------+--------+------------+-----------------+-----------------+-----------+-------------------------+
| Masculine |  4274  |    934     |       804       |       115       |    503    |            5            |
|  Feminine |  4274  |     21     |        14       |        5        |    310    |           209           |
|  Neutral  |  183   |     37     |        36       |        0        |     7     |      to M:3, to F:0     |
+-----------+--------+------------+-----------------+-----------------+-----------+-------------------------+


In [5]:
t2 = PrettyTable(['-de-',"Occup.", 'Wiki pages', "Redirects", "No page"])
t2.add_row(['Masculine',len(words), len(page_m)+len(m_page_levenshtein), len(m_redirection),
            len(no_page_m)])
t2.add_row(['Feminine',len(words_revert), len(page_f),
            len(f_redirection),
            len(no_page_f)])
t2.add_row(['Neutral',len(neutral), len(page_n),len(redirection_n),len(no_page_n)])
print t2
print
t3 = PrettyTable(['-de-','Wiki pages', "Validated pages","Ambigious pages","Not validated pages"])
t3.add_row(['Masculine', len(page_m)+len(m_page_levenshtein),
            len(page_m_validated)+len(m_page_levenshtein),
            len(page_m_ambigious),len(page_m_other)])
t3.add_row(['Feminine', len(page_f), len(page_f_validated), len(page_f_ambigious),len(page_f_other)])
t3.add_row(['Neutral', len(page_n), len(page_n_validated), len(page_n_ambigious),len(page_n_other)])

print t3
print
t4 = PrettyTable(['--de--',"Redirects","Redirects to oposit gender", "Other redirects"])
t4.add_row(['Masculine',len(m_redirection),
            len(m_links_to_feminine),len(m_links_to_smth)])
t4.add_row(['Feminine',len(f_redirection),
            len(f_links_to_mascuilne)+len(f_links_to_mascuilne_lev)+len(f_links_to_mascuilne_with_link),len(f_links_to_smth)])
t4.add_row(['Neutral',len(redirection_n),"to M:"+str(len(n_links_to_masculine))+", to F:0",len(n_links_to_smth)])
print t4


+-----------+--------+------------+-----------+---------+
|    -de-   | Occup. | Wiki pages | Redirects | No page |
+-----------+--------+------------+-----------+---------+
| Masculine |  4274  |    934     |    503    |   2837  |
|  Feminine |  4274  |     21     |    310    |   3943  |
|  Neutral  |  183   |     37     |     7     |   139   |
+-----------+--------+------------+-----------+---------+

+-----------+------------+-----------------+-----------------+---------------------+
|    -de-   | Wiki pages | Validated pages | Ambigious pages | Not validated pages |
+-----------+------------+-----------------+-----------------+---------------------+
| Masculine |    934     |       804       |       115       |          15         |
|  Feminine |     21     |        14       |        5        |          2          |
|  Neutral  |     37     |        36       |        0        |          1          |
+-----------+------------+-----------------+-----------------+---------------------

# Look into other_redirects:

In [6]:
#Check whether its male label is sinonim or it is a field name
fields=load_simple_json('de/field.json')
f_links_to_field={}
f_links_to_realy_smth_else={}
f_links_to_masculine_that_already_counted={}

k=0
for i in f_links_to_smth:
    if f_links_to_smth[i] in fields:
        f_links_to_field[i]=f_links_to_smth[i]
    elif words.has_key(f_links_to_smth[i]):
        if f_links_to_smth[i] in page_m:
            #print i," links to profesion that we already count:", f_links_to_smth[i]
            f_links_to_masculine_that_already_counted[i]=f_links_to_smth[i]
    else:
        k+=1
        f_links_to_realy_smth_else[i]=f_links_to_smth[i]
print "# of female prof. labels redirected to already counted male prof.=",len(f_links_to_masculine_that_already_counted)
print "# of female prof. labels that links to smth else",k
print "TODO: Work manually on te list:"
f_links_to_realy_smth_else

# of female prof. labels redirected to already counted male prof.= 17
# of female prof. labels that links to smth else 55
TODO: Work manually on te list:


{u'Akrobatin': u'Akrobatik',
 u'Ansagerin': u'Programmsprecher',
 u'Astrologin': u'Astrologie',
 u'Betriebs\xf6konomin': u'Betriebswirtschaftslehre',
 u'Bildende K\xfcnstlerin': u'Bildende Kunst',
 u'Bildhauerin': u'Bildhauerei',
 u'Blumenbinderin': u'Floristik (Handwerk)',
 u'DJane': u'DJ [[Kategorie:Abk\xfcrzung|DJANE',
 u'Einzelhandelskauffrau': u'Kaufmann im Einzelhandel',
 u'Entertainerin': u'Unterhaltungsk\xfcnstler',
 u'Fachkrankenschwester': u'Fachgesundheits- und Krankenpfleger',
 u'Familientherapeutin': u'Familientherapie',
 u'Fernsehmoderatorin': u'Moderator (Beruf)',
 u'Fu\xdfballspielerin': u'Frauenfu\xdfball',
 u'Garderobiere': u'Garderobe (Raum)',
 u'Gemeindeschwester': u'Pflegefachkraft in der ambulanten Pflege',
 u'Gesch\xe4ftsf\xfchrerin': u'Gesch\xe4ftsf\xfchrung',
 u'Gro\xdfhandelskauffrau': u'Kaufmann im Gro\xdf- und Au\xdfenhandel',
 u'Heilerziehungspflegehelferin': u'Heilerziehungspflege',
 u'Holzbildhauerin': u'Bildschnitzer',
 u'J\xe4gerin': u'j\xe4ger',
 u'Kab